In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession 
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import json

In [2]:
ss = SparkSession.Builder() \
     .appName("Realtime") \
     .master("spark://spark-master-1:7077") \
     .config("spark.jars", "./spark-streaming-kafka-0-8-assembly_2.11-2.0.0-preview.jar") \
     .getOrCreate()

In [3]:
sc = ss.sparkContext
ssc = StreamingContext(sc, 5)
ss.sparkContext.setLogLevel('WARN')

In [4]:
brokers = 'kafka-1:9092,kafka-2:9092'
topic='BTC'

In [5]:
ks = KafkaUtils.createDirectStream(
    ssc, [topic], kafkaParams={"metadata.broker.list":brokers})

In [6]:
def insert_row(x):
    if x is None or len(x)<1:
        return
    data_list = x.split(',')
    from pymongo import MongoClient
    conn = MongoClient('mongodb', 27017)
    db = conn.btc
    db.realtime.insert({
        'timestamp': data_list[0],
        'buy_price': data_list[1],
        'sell_price': data_list[2]
    })
    conn.close()

In [ ]:
lines = ks.map(
    lambda x: '{},{},{}'.format(
        json.loads(x[1])['timestamp'],
        json.loads(x[1])['buy_price'],
        json.loads(x[1])['sell_price']
    )
)
lines.foreachRDD(lambda rdd: rdd.foreach(insert_row))

ssc.start()
ssc.awaitTermination()